<b>Introduction: Kaela Coffee Company<b>

Established in 2008, Kaela Coffee Company (KCC) is a locally-owned coffee shop located in Miami, Florida. KCC primarily sells iced-coffees and an assortment of baked goods. Annual sales continue to increase year-over-year, and profits continue to set new record highs. After discussion with board members on the topic of increasing sales and profits, a market development strategy was selected with the goal of expanding one new location outside of the southern Florida location. After further discussion and analysis, San Francisco, California was selected as the target location to open a retail location, duplicating the operating model of the Miami KCC operation.

Before moving forward with development plans in San Francisco, and in an effort to make a good strategic decision on the new location, the board members are requesting an analysis report. The company requests an analysis of retail venues currently operating in San Francisco. Additionally, the company is requesting details on primary competitors in the San Francisco area. Finally, board members also request an official recommendation on which neighborhood to introduce the new retail location in. The goal opening date for the new retail location is May 2020.

<b>Description of the Data</b>

Research and analysis will be conducted with Foursquare location and venue data. This data set will be vital to highlight the current retail operations in the San Francisco locations under consideration. Foursquare is a technology company that uses location intelligence to build meaningful consumer experiences and business solutions.

<b>Data Analysis</b>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [3]:
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


print('Libraries imported!')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  23.70 MB/s
geopy-1.18.1-p 100% |################################| Time: 0:00:00  35.31 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  56.87 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  33.96 MB/s
vincent-0.4.4- 100% |###################

In [4]:
# Access the data
!wget -q -O 'sanfran_data.json' https://cocl.us/sanfran_geojson
print('Data downloaded!')

Data downloaded!


In [5]:
world_geo = r'sanfran_data.json' # geojson file
# San Francisco latitude and longitude values
latitude = 37.77
longitude = -122.42
# create map and display it
sanfran_map = folium.Map(location=[latitude, longitude], zoom_start=12)

# display the map of San Francisco
sanfran_map

In [10]:
# Explore first neighborhood in dataframe.
df_data_1.loc[0, 'District']

'Bayview'

In [11]:
# Get the neighborhood's latitude and longitude values.
neighborhood_latitude = df_data_1.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_data_1.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_data_1.loc[0, 'District'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Bayview are 37.73093607, -122.37389040000001.


In [30]:
# Send the GET request and examine the results
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c6ca13a4c1f67636da946ab'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4f3709abe4b0aa62a1d35520-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/spa_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1ed941735',
         'name': 'Spa',
         'pluralName': 'Spas',
         'primary': True,
         'shortName': 'Spa'}],
       'id': '4f3709abe4b0aa62a1d35520',
       'location': {'address': '748 Innes Ave',
        'cc': 'US',
        'city': 'San Francisco',
        'country': 'United States',
        'distance': 90,
        'formattedAddress': ['748 Innes Ave',
         'San Francisco, CA 94124',
         'United States'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 37.730525422729094,
  

In [31]:
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

In [32]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [33]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Archimedes Banya,Spa,37.730525,-122.372998
1,Flora Grubb Gardens,Flower Shop,37.739589,-122.390171
2,Pier 92,Building,37.746498,-122.383486
3,Minnesota Street Projects Gallery,Art Gallery,37.754247,-122.389542
4,Dogpatch Boulders,Climbing Gym,37.756625,-122.387970


In [34]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [35]:
# Let's find how many unique categories from all returned venues
print('There are {} uniques categories.'.format(len(nearby_venues['categories'].unique())))

There are 58 uniques categories.


In [36]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [37]:
sanfran_venues = getNearbyVenues(names=df_data_1['District'],
                                   latitudes=df_data_1['Latitude'],
                                   longitudes=df_data_1['Longitude']
                                  )

Bayview
Central
Ingleside
Mission
Northern
Park
Richmond
Southern
Taraval
Tenderloin


In [38]:
print(sanfran_venues.shape)
sanfran_venues.head()

(495, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bayview,37.730936,-122.37389,Archimedes Banya,37.730525,-122.372998,Spa
1,Bayview,37.730936,-122.37389,International Safe,37.732482,-122.374878,Warehouse Store
2,Bayview,37.730936,-122.37389,"Arelious Walker Mosaic Staircase ""Flights of F...",37.730728,-122.374018,Public Art
3,Bayview,37.730936,-122.37389,Maya Manor,37.729212,-122.371846,Jewelry Store
4,Bayview,37.730936,-122.37389,India Basin,37.734312,-122.376112,Park


In [39]:
sanfran_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bayview,5,5,5,5,5,5
Central,100,100,100,100,100,100
Ingleside,27,27,27,27,27,27
Mission,100,100,100,100,100,100
Northern,100,100,100,100,100,100
Park,44,44,44,44,44,44
Richmond,15,15,15,15,15,15
Southern,2,2,2,2,2,2
Taraval,2,2,2,2,2,2


In [40]:
print('There are {} uniques categories.'.format(len(sanfran_venues['Venue Category'].unique())))

There are 170 uniques categories.


In [41]:
# Analyze each district
# one hot encoding
sanfran_onehot = pd.get_dummies(sanfran_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
sanfran_onehot['Neighborhood'] = sanfran_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [sanfran_onehot.columns[-1]] + list(sanfran_onehot.columns[:-1])
sanfran_onehot = sanfran_onehot[fixed_columns]

sanfran_onehot.head()

,Neighborhood,Accessories Store,American Restaurant,Antique Shop,Arepa Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Trail,Used Bookstore,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Winery,Yoga Studio
0,Bayview,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Bayview,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,Bayview,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Bayview,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Bayview,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
sanfran_onehot.shape

(495, 171)

In [43]:
sanfran_grouped = sanfran_onehot.groupby('Neighborhood').mean().reset_index()
sanfran_grouped

,Neighborhood,Accessories Store,American Restaurant,Antique Shop,Arepa Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Trail,Used Bookstore,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Winery,Yoga Studio
0,Bayview,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.0,0.00,...,0.000000,0.00,0.00,0.000000,0.2,0.00,0.00,0.00,0.00,0.00
1,Central,0.000000,0.010000,0.00,0.00,0.00,0.000000,0.000000,0.0,0.00,...,0.000000,0.00,0.00,0.010000,0.0,0.01,0.01,0.01,0.00,0.02
2,Ingleside,0.000000,0.037037,0.00,0.00,0.00,0.000000,0.000000,0.0,0.00,...,0.000000,0.00,0.00,0.111111,0.0,0.00,0.00,0.00,0.00,0.00
3,Mission,0.000000,0.000000,0.01,0.01,0.01,0.000000,0.000000,0.0,0.01,...,0.000000,0.00,0.00,0.000000,0.0,0.02,0.00,0.00,0.00,0.03
4,Northern,0.000000,0.020000,0.00,0.00,0.00,0.010000,0.010000,0.0,0.00,...,0.000000,0.00,0.03,0.070000,0.0,0.00,0.03,0.01,0.00,0.00
5,Park,0.022727,0.000000,0.00,0.00,0.00,0.000000,0.022727,0.0,0.00,...,0.045455,0.00,0.00,0.000000,0.0,0.00,0.00,0.00,0.00,0.00
6,Richmond,0.000000,0.000000,0.00,0.00,0.00,0.066667,0.000000,0.0,0.00,...,0.066667,0.00,0.00,0.000000,0.0,0.00,0.00,0.00,0.00,0.00
7,Southern,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.5,0.00,...,0.000000,0.00,0.00,0.000000,0.0,0.00,0.00,0.00,0.00,0.00
8,Taraval,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.0,0.00,...,0.000000,0.00,0.00,0.000000,0.0,0.00,0.00,0.00,0.00,0.00
9,Tenderloin,0.000000,0.000000,0.00,0.00,0.03,0.000000,0.000000,0.0,0.00,...,0.000000,0.01,0.00,0.020000,0.0,0.00,0.03,0.00,0.01,0.00


In [44]:
sanfran_grouped.shape

(10, 171)

In [45]:
num_top_venues = 10

for hood in sanfran_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = sanfran_grouped[sanfran_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bayview----
             venue  freq
0              Spa   0.2
1       Public Art   0.2
2  Warehouse Store   0.2
3    Jewelry Store   0.2
4             Park   0.2
5      Opera House   0.0
6  Motorcycle Shop   0.0
7      Music Store   0.0
8      Music Venue   0.0
9       Nail Salon   0.0


----Central----
                     venue  freq
0              Coffee Shop  0.08
1              Men's Store  0.04
2                      Gym  0.04
3           Sandwich Place  0.04
4         Sushi Restaurant  0.04
5       Italian Restaurant  0.03
6      Japanese Restaurant  0.03
7  New American Restaurant  0.03
8                     Café  0.03
9                    Hotel  0.02


----Ingleside----
                       venue  freq
0                Pizza Place  0.11
1      Vietnamese Restaurant  0.11
2         Mexican Restaurant  0.07
3  Latin American Restaurant  0.07
4               Liquor Store  0.07
5                Bus Station  0.07
6                Coffee Shop  0.04
7        Filipino Restaurant

In [46]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [47]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = sanfran_grouped['Neighborhood']

for ind in np.arange(sanfran_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(sanfran_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bayview,Public Art,Warehouse Store,Spa,Park,Jewelry Store,Convenience Store,Concert Hall,Food Truck,Flower Shop,Filipino Restaurant
1,Central,Coffee Shop,Men's Store,Sushi Restaurant,Sandwich Place,Gym,New American Restaurant,Japanese Restaurant,Italian Restaurant,Café,Seafood Restaurant
2,Ingleside,Vietnamese Restaurant,Pizza Place,Bus Station,Mexican Restaurant,Liquor Store,Latin American Restaurant,Pool Hall,Thai Restaurant,Burrito Place,Filipino Restaurant
3,Mission,Boutique,Pizza Place,Gym / Fitness Center,Mexican Restaurant,Coffee Shop,Sushi Restaurant,Yoga Studio,Furniture / Home Store,Bar,Beer Bar
4,Northern,Vietnamese Restaurant,Café,French Restaurant,Coffee Shop,Furniture / Home Store,Beer Bar,Sandwich Place,Park,Thai Restaurant,Vegetarian / Vegan Restaurant


In [48]:
# create map of Manhattan using latitude and longitude values
map_san = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_data_1['Latitude'], df_data_1['Longitude'], df_data_1['District']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_san)  
# add pop-up text to each marker on the map
latitudes = list(df_data_1.Latitude)
longitudes = list(df_data_1.Longitude)
labels = list(df_data_1.District)

for lat, lng, label in zip(latitudes, longitudes, labels):
    folium.Marker([lat, lng], popup=label).add_to(map_san)    
    
map_san

In [49]:
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bayview,Public Art,Warehouse Store,Spa,Park,Jewelry Store,Convenience Store,Concert Hall,Food Truck,Flower Shop,Filipino Restaurant
1,Central,Coffee Shop,Men's Store,Sushi Restaurant,Sandwich Place,Gym,New American Restaurant,Japanese Restaurant,Italian Restaurant,Café,Seafood Restaurant
2,Ingleside,Vietnamese Restaurant,Pizza Place,Bus Station,Mexican Restaurant,Liquor Store,Latin American Restaurant,Pool Hall,Thai Restaurant,Burrito Place,Filipino Restaurant
3,Mission,Boutique,Pizza Place,Gym / Fitness Center,Mexican Restaurant,Coffee Shop,Sushi Restaurant,Yoga Studio,Furniture / Home Store,Bar,Beer Bar
4,Northern,Vietnamese Restaurant,Café,French Restaurant,Coffee Shop,Furniture / Home Store,Beer Bar,Sandwich Place,Park,Thai Restaurant,Vegetarian / Vegan Restaurant
5,Park,Park,Café,Pizza Place,Thai Restaurant,Garden,Sushi Restaurant,Trail,Deli / Bodega,Rental Car Location,Candy Store
6,Richmond,Pharmacy,Bus Stop,Sculpture Garden,Exhibit,Golf Course,Park,Monument / Landmark,Outdoor Sculpture,Art Museum,Trail
7,Southern,Breakfast Spot,Athletics & Sports,Yoga Studio,Ethiopian Restaurant,Furniture / Home Store,French Restaurant,Food Truck,Flower Shop,Filipino Restaurant,Exhibit
8,Taraval,Clothing Store,Bus Stop,Yoga Studio,Donut Shop,French Restaurant,Food Truck,Flower Shop,Filipino Restaurant,Exhibit,Event Space
9,Tenderloin,Hotel,Theater,Cocktail Bar,Speakeasy,Indian Restaurant,Music Venue,Jazz Club,Hostel,Bar,Thai Restaurant


In [50]:
address = 'San Francisco, California'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

37.7792808 -122.4192363


In [51]:
search_query = 'coffee'
radius = 8047
print(search_query + ' .... OK!')

coffee .... OK!


In [54]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,...,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'primary': True, 'pluralName': 'Coffee Shops...",NaN,NaN,NaN,NaN,NaN,NaN,False,5560dbdb498e91a2bcde84f6,1355 Market St Ste 190,...,"[1355 Market St Ste 190 (at 10th St), San Fran...","[{'label': 'display', 'lat': 37.77628641647586...",37.776286,-122.416867,NaN,94103,CA,Blue Bottle Coffee,v-1550623079,NaN
1,"[{'primary': True, 'pluralName': 'Coffee Shops...",NaN,NaN,NaN,NaN,NaN,NaN,False,4dd94350e4cd37c893d8146f,432B Octavia St,...,"[432B Octavia St (at Hayes St), San Francisco,...","[{'label': 'display', 'lat': 37.77647634472176...",37.776476,-122.424281,NaN,94102,CA,Ritual Coffee Roasters,v-1550623079,52354550
2,"[{'primary': True, 'pluralName': 'Coffee Shops...",NaN,NaN,NaN,NaN,NaN,NaN,False,43d3901ef964a5201f2e1fe3,315 Linden St,...,"[315 Linden St (btwn Gough & Octavia St), San ...","[{'label': 'display', 'lat': 37.77643013182072...",37.776430,-122.423224,Hayes Valley,94102,CA,Blue Bottle Coffee,v-1550623079,NaN
3,"[{'primary': True, 'pluralName': 'Coffee Shops...",NaN,NaN,NaN,NaN,NaN,NaN,False,4a4b8a43f964a5206fac1fe3,270 7th St,...,"[270 7th St (btwn Howard & Folsom St), San Fra...","[{'label': 'display', 'lat': 37.77700097623915...",37.777001,-122.408519,NaN,94103,CA,Sightglass Coffee,v-1550623079,48020594
4,"[{'primary': True, 'pluralName': 'Coffee Shops...",NaN,NaN,NaN,NaN,NaN,NaN,False,5151a10ce4b06ae7735335db,399 Golden Gate Ave,...,"[399 Golden Gate Ave (Larkin), San Francisco, ...","[{'label': 'display', 'lat': 37.78143271775249...",37.781433,-122.417073,NaN,94102,CA,Philz Coffee,v-1550623079,NaN


In [55]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Blue Bottle Coffee,Coffee Shop,1355 Market St Ste 190,US,San Francisco,United States,at 10th St,393,"[1355 Market St Ste 190 (at 10th St), San Fran...","[{'label': 'display', 'lat': 37.77628641647586...",37.776286,-122.416867,NaN,94103,CA,5560dbdb498e91a2bcde84f6
1,Ritual Coffee Roasters,Coffee Shop,432B Octavia St,US,San Francisco,United States,at Hayes St,542,"[432B Octavia St (at Hayes St), San Francisco,...","[{'label': 'display', 'lat': 37.77647634472176...",37.776476,-122.424281,NaN,94102,CA,4dd94350e4cd37c893d8146f
2,Blue Bottle Coffee,Coffee Shop,315 Linden St,US,San Francisco,United States,btwn Gough & Octavia St,473,"[315 Linden St (btwn Gough & Octavia St), San ...","[{'label': 'display', 'lat': 37.77643013182072...",37.776430,-122.423224,Hayes Valley,94102,CA,43d3901ef964a5201f2e1fe3
3,Sightglass Coffee,Coffee Shop,270 7th St,US,San Francisco,United States,btwn Howard & Folsom St,976,"[270 7th St (btwn Howard & Folsom St), San Fra...","[{'label': 'display', 'lat': 37.77700097623915...",37.777001,-122.408519,NaN,94103,CA,4a4b8a43f964a5206fac1fe3
4,Philz Coffee,Coffee Shop,399 Golden Gate Ave,US,San Francisco,United States,Larkin,305,"[399 Golden Gate Ave (Larkin), San Francisco, ...","[{'label': 'display', 'lat': 37.78143271775249...",37.781433,-122.417073,NaN,94102,CA,5151a10ce4b06ae7735335db
5,Blue Bottle Coffee,Coffee Shop,66 Mint Plaza,US,San Francisco,United States,at Jessie St,1076,"[66 Mint Plaza (at Jessie St), San Francisco, ...","[{'label': 'display', 'lat': 37.78258403122913...",37.782584,-122.407743,NaN,94103,CA,49ca8f4df964a520b9581fe3
6,Four Barrel Coffee,Coffee Shop,375 Valencia St,US,San Francisco,United States,btwn 14th St & 15th St,1387,"[375 Valencia St (btwn 14th St & 15th St), San...","[{'label': 'display', 'lat': 37.76701253204111...",37.767013,-122.422010,Mission District,94103,CA,480d1a5ef964a520284f1fe3
7,Andytown Coffee,Coffee Shop,NaN,US,San Francisco,United States,NaN,538,"[San Francisco, CA 94103, United States]","[{'label': 'display', 'lat': 37.77458921534537...",37.774589,-122.417744,NaN,94103,CA,59370864bfc6d00154644d69
8,Verve Coffee Roasters,Coffee Shop,2101 Market St,US,San Francisco,United States,at Church St,1610,"[2101 Market St (at Church St), San Francisco,...","[{'label': 'display', 'lat': 37.767121, 'lng':...",37.767121,-122.429152,NaN,94114,CA,589f6a4372714f7d35566b6a
9,Peet's Coffee & Tea,Coffee Shop,"601 Van Ness,",US,San Francisco,United States,NaN,307,"[601 Van Ness,, San Francisco, CA 94102, Unite...","[{'label': 'display', 'lat': 37.78172207, 'lng...",37.781722,-122.420871,NaN,94102,CA,49ee7fe2f964a52064681fe3


In [56]:
dataframe_filtered.name

0                    Blue Bottle Coffee
1                Ritual Coffee Roasters
2                    Blue Bottle Coffee
3                     Sightglass Coffee
4                          Philz Coffee
5                    Blue Bottle Coffee
6                    Four Barrel Coffee
7                       Andytown Coffee
8                 Verve Coffee Roasters
9                   Peet's Coffee & Tea
10                  Peet's Coffee & Tea
11    Ma’Velous Coffee & Little Griddle
12                        Peet's Coffee
13               Ritual Coffee Roasters
14                         Philz Coffee
15              Sextant Coffee Roasters
16                   Blue Bottle Coffee
17                   Blue Bottle Coffee
18                         Philz Coffee
19               Cup-A-Joe Coffee House
20                Celtic Coffee Company
21                It's Tops Coffee Shop
22                           Coffee Bar
23                  Peet's Coffee & Tea
24                  Réveille Coffee Co.


In [57]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Center

# add the coffee spots as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        popup=label,
        fill = True,
        fill_color='green',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [58]:
venues_map2 = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel


# add the coffee spots as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='green',
        popup=label,
        fill = True,
        fill_color='green',
        fill_opacity=0.6
    ).add_to(venues_map2)

# add markers to map
for lat, lng, label in zip(df_data_1['Latitude'], df_data_1['Longitude'], df_data_1['District']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(venues_map2)  
# add pop-up text to each marker on the map
latitudes = list(df_data_1.Latitude)
longitudes = list(df_data_1.Longitude)
labels = list(df_data_1.District)

for lat, lng, label in zip(latitudes, longitudes, labels):
    folium.Marker([lat, lng], popup=label).add_to(venues_map2)
# display map
venues_map2

<b>Methodology</b>

Foursquare data was used to analyze the ten districts in San Francisco that were under consideration: Central, Mission, Northern, Park, Tenderloin, Bayview, Richmond, Taraval, Ingleside, and Southern. For each region, the top ten category of venue was identified for each district. Additionally, all coffee establishments within a five mile radius of the city were identified. Within these establishment, top competitors are highlighted as an additional business risk.

<b>Results</b>

Four of the ten districts have coffee venues within their respective top ten venue type by frequency. Central District has the highest frequency of coffee venues out of the ten districts. Within the five-mile radius, 49 coffee establishments were identified. Of these 49 venues, there are a few noteworthy competitors: Peet's Coffee, Blue Bottle Coffee, Ritual Coffee Roasters, and Philz Coffee.

<b>Discussion</b>

Coffee establishment activity is currently very high in many locations within San Francisco. However, there is intense competition in this arena. Of the ten districts under analysis, four were identified as having a high level of coffee vendors currently in operation. Evidence suggests that the establishments with comfortable venue locations and attractive prices see a steady flow of customers in high-traffic districts. Of particular note, four districts fall into this category: Central, Mission, Northern, and Tenderloin. Of these four, Tenderloin district is unique in that it has a slightly less density of coffee venues compared to the other three districts.

Of the four districts identified as high-volume locations, two have an additional benefit of being close in proximity to the Financial District - the Central and Tenderloin Districts. These locations experience an especially high volume of traffic during morning and lunch hour in addition to other prime customer activity times such as the weekends. It is worthwhile to note that this type of environment is what the firm currently operates in in the Miami, Florida location.

<b>Conclusion</b>

Based on current economic activity, slightly less density of current coffee venues, and similarity with the current Miami location, the recommended location for new Kaela Coffee Company operation is in the Tenderloin District of San Francisco.